# Best X finder - Test for one period

Veremos cómo funciona el algoritmo en un script distinto puesto que podría haber variables inicializadas en los test del script *two_best_X_finder.ipynb*

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from termcolor import colored
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent/Best_X_finder


In [4]:
print("Para modificar el tamaño de todos los gráficos")
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,9

Para modificar el tamaño de todos los gráficos


In [5]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Exploración de datos

Estudiaremos de manera general si hay datos faltantes, así como el tipo de datos que contiene

In [6]:
# Años de análisis
starting_year_t = 2011
period = 11
csv = '.csv'
# Directorios
pitchers_path_t = 'ETL_data/Period_t/Free_Agent/Pitchers/free_agents_pitchers_'
hitters_path_t = 'ETL_data/Period_t/Free_Agent/Hitters/free_agents_batters_'
# Originales:
df_pitchers_t = [None]*period
df_pitchers_t_copy = [None]*period
df_hitters_t = [None]*period
df_hitters_t_copy = [None]*period

Leamos todos las bases de datos correspondientes

In [7]:
for year in range(0,period):
    df_pitchers_t[year] = pd.read_csv(pitchers_path_t + str(starting_year_t + year) + csv)
    df_pitchers_t_copy[year] = df_pitchers_t[year].copy()
    
    df_hitters_t[year] = pd.read_csv(hitters_path_t + str(starting_year_t + year) + csv)
    df_hitters_t_copy[year] = df_hitters_t[year].copy()

Observemos el contenido de las bases de datos

In [8]:
# Pitchers
df_pitchers_t_copy[5].head()

,Acronimo,Altura,Anio_x,Anio_y,Anios_contrato,Bateos_pitcher,Bateos_pitcher_2,Cantidad de equipos,Cantidad_agentes_libres,Carreras,...,WHIP,WHIP_2,WS ganadas,Walks,Walks_2,Wins,Wins_2,ln_Sueldo,ln_Sueldo_base,ln_Sueldo_regular
0,TEX,6.5,2016,6,1,116,13456,2,10,68,...,1.36,1.8496,0,46,2116,7,49,13.137252,13.137252,13.109548
1,CIN,6.6,2016,6,1,89,7921,2,8,64,...,2.05,4.2025,5,31,961,2,4,14.508658,14.508658,14.508658
2,DET,6.0,2013,6,5,171,29241,1,11,108,...,1.46,2.1316,4,53,2809,7,49,16.636889,16.588099,16.636889
3,MIL,6.3,2016,6,1,80,6400,1,8,30,...,1.47,2.1609,0,17,289,2,4,13.764217,13.764217,13.764217
4,CIN,6.5,2016,6,1,72,5184,2,8,38,...,1.44,2.0736,5,38,1444,6,36,13.304685,13.304685,13.304685


In [9]:
list(enumerate(df_pitchers_t_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio_x'),
 (3, 'Anio_y'),
 (4, 'Anios_contrato'),
 (5, 'Bateos_pitcher'),
 (6, 'Bateos_pitcher_2'),
 (7, 'Cantidad de equipos'),
 (8, 'Cantidad_agentes_libres'),
 (9, 'Carreras'),
 (10, 'Carreras_2'),
 (11, 'Carreras_ganadas'),
 (12, 'Carreras_ganadas_2'),
 (13, 'Comando'),
 (14, 'Comando_2'),
 (15, 'Control'),
 (16, 'Control_2'),
 (17, 'Dominio'),
 (18, 'Dominio_2'),
 (19, 'ERA'),
 (20, 'ERA_2'),
 (21, 'Edad'),
 (22, 'Equipo'),
 (23, 'Equipo_anterior'),
 (24, 'Estado'),
 (25, 'Inning_pitched'),
 (26, 'Inning_pitched_2'),
 (27, 'Juegos'),
 (28, 'Juegos totales'),
 (29, 'Juegos_2'),
 (30, 'Juegos_iniciados'),
 (31, 'Juegos_iniciados_2'),
 (32, 'Jugador'),
 (33, 'Losses'),
 (34, 'Losses_2'),
 (35, 'Pennants won'),
 (36, 'Peso'),
 (37, 'Playoffs'),
 (38, 'Posicion'),
 (39, 'Promedio_victorias'),
 (40, 'Saves'),
 (41, 'Saves_2'),
 (42, 'Status'),
 (43, 'Strike-outs'),
 (44, 'Strike-outs_2'),
 (45, 'Sueldo'),
 (46, 'Sueldo_base'),
 (47, 'Sueldo_regula

In [10]:
# Hitters
df_hitters_t_copy[5].head()

,Acronimo,Altura,Anio_x,Anio_y,Anios_contrato,At-bats,At-bats_2,Bateos,Bateos_2,Cantidad de equipos,...,Valor_contrato,Valor_contrato_total,Valor_promedio_contrato,Victorias,WAR,WAR_2,WS ganadas,ln_Sueldo,ln_Sueldo_base,ln_Sueldo_regular
0,TEX,6.5,2016,6,1,2,4,0,0,2,...,507500,20165000,507500,95,0.93,0.8649,0,13.137252,13.137252,13.109548
1,ATL,6.3,2016,6,1,247,61009,55,3025,1,...,3000000,19157500,3000000,67,-1.28,1.6384,3,14.930652,14.508658,14.930652
2,LAA,6.3,2012,6,10,593,351649,159,25281,5,...,240000000,12525000,24000000,73,1.29,1.6641,1,17.034386,17.034386,17.034386
3,NYM,6.0,2016,6,1,234,54756,48,2304,2,...,5750000,48757500,5750000,87,-0.38,0.1444,2,15.564710,15.564710,15.564710
4,KC,6.1,2016,6,4,445,198025,98,9604,2,...,72000000,186750000,18000000,81,0.91,0.8281,2,16.300417,16.300417,16.300417


Veamos las columnas con sus índices

In [11]:
list(enumerate(df_hitters_t_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio_x'),
 (3, 'Anio_y'),
 (4, 'Anios_contrato'),
 (5, 'At-bats'),
 (6, 'At-bats_2'),
 (7, 'Bateos'),
 (8, 'Bateos_2'),
 (9, 'Cantidad de equipos'),
 (10, 'Cantidad_agentes_libres'),
 (11, 'Edad'),
 (12, 'Equipo'),
 (13, 'Equipo_anterior'),
 (14, 'Estado'),
 (15, 'Home-runs'),
 (16, 'Home-runs_2'),
 (17, 'Juegos'),
 (18, 'Juegos totales'),
 (19, 'Juegos_2'),
 (20, 'Jugador'),
 (21, 'OPS'),
 (22, 'OPS_2'),
 (23, 'Pennants won'),
 (24, 'Peso'),
 (25, 'Playoffs'),
 (26, 'Porcentaje_bateo'),
 (27, 'Porcentaje_bateo_2'),
 (28, 'Porcetnaje_juegos'),
 (29, 'Porcetnaje_juegos_2'),
 (30, 'Posicion'),
 (31, 'Promedio_victorias'),
 (32, 'RBI'),
 (33, 'RBI_2'),
 (34, 'Status'),
 (35, 'Sueldo'),
 (36, 'Sueldo_base'),
 (37, 'Sueldo_regular'),
 (38, 'TVS'),
 (39, 'TVS_2'),
 (40, 'Valor_contrato'),
 (41, 'Valor_contrato_total'),
 (42, 'Valor_promedio_contrato'),
 (43, 'Victorias'),
 (44, 'WAR'),
 (45, 'WAR_2'),
 (46, 'WS ganadas'),
 (47, 'ln_Sueldo'),
 (48, 'ln_

Veamos si hay bases de datos con variables que distintas entre sí

## Test

In [12]:
def etl_regression(X_raw, Y_raw, df_copy):
    # Creación de la base de datos a transformar las variables de la regresión
    jugador = 'Jugador'
    YX_raw = [None]*period
    
    for year in range(0, period):
        YX_raw[year] = df_copy[year][[jugador, Y_raw, X_raw]]

    # Ajuste de inflación en los salarios    
    change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46,
                                   2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86,
                                   2019:5.99, 2020:4.70, 2021:1}
    for year in range(0, period):
        YX_raw[year][[Y_raw]] = YX_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
        YX_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
        YX_raw[year].rename(columns = {X_raw: X_raw + '_' + str(2011 + year)}, inplace = True)

    # Filtrado de base de datos
    YX_panel_raw = pd.merge(YX_raw[0], YX_raw[1], how = 'outer', on = 'Jugador')

    for year in range(2,period):
        YX_panel_raw = pd.merge(YX_panel_raw, YX_raw[year], how = 'outer', on = 'Jugador')

    YX_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw.sort_index(axis = 1, inplace = True)

    # Recolocación al principio de Jugador
    player_column = YX_panel_raw.pop('Jugador')
    YX_panel_raw.insert(0, 'Jugador', player_column)

    # Creación de base de datos auxiliares  para remover los NaN
    split_column = 1 + period
    max_nan = 2

    if Y_raw > X_raw:
        X_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]

    nulls_quantity = period - max_nan
    X_drop_etl = X_drop_etl.iloc[X_drop_etl[(X_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    X_index_drop = X_drop_etl.index

    if Y_raw > X_raw:
        Y_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    Y_drop_etl = Y_drop_etl.iloc[Y_drop_etl[(Y_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    Y_index_drop = Y_drop_etl.index

    # índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
    index_drop = list(set(X_index_drop).union(set(Y_index_drop)))

    YX_panel_raw.drop(index = index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)

    # Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
    if Y_raw > X_raw:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, 1:split_column].max(axis = 1)
    else:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]].max(axis = 1)
    YX_max_0 = YX_panel_raw[YX_panel_raw[X_raw + '_max'] == 0]
    YX_max_0_index_drop = YX_max_0.index
    YX_panel_raw.drop(columns = X_raw + '_max', inplace = True)
    YX_panel_raw.drop(index = YX_max_0_index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)

    # Variable X
    if Y_raw > X_raw:
        X_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    max_element = X_etl.max(axis = 1)
    min_element = X_etl.min(axis = 1)
    max_min = (max_element + min_element)/2
    dummy_names = []
    X_names = []
    for year in range(0,period):
        dummy_names.append("I_" + str(2011 + year))
        X_names.append("X_" + str(2011 + year))
    for year in range(0,period):
        X_conditions = [
        (X_etl.iloc[:,year] == max_element) & ~(X_etl.iloc[:,year].isnull()),
        (X_etl.iloc[:,year] != max_element) & ~(X_etl.iloc[:,year].isnull())
        ]
        X_conditions_values = [0,1]
        X_etl[dummy_names[year]] = np.select(X_conditions, X_conditions_values, default = np.nan)
    X_aux = X_etl.iloc[:,:period]
    Dummy = X_etl.iloc[:,period:]
    X_max = X_aux.max(axis = 1)
    X_aux[['Max']] = np.sqrt(X_max)
    for year in range(0,period):    
        X_etl['X_' + str(2011 + year)] = \
        (X_aux[X_raw + '_' + str(2011 + year)]*
        ((-1)**Dummy['I_' + str(2011 + year)]))
    X_etl = X_etl.div(X_aux['Max'].values, axis = 0)
    X_etl = X_etl.iloc[:,2*period:]
    for i in range(0,X_etl.shape[0]):
        row = X_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            X_etl.iloc[i,j] = sustitute[j]

    X_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                     'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
    X_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    X_player_column = X_etl.pop('Jugador')
    X_etl.insert(0, 'Jugador', X_player_column)

    # Variable Y
    if Y_raw > X_raw:
        Y_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_etl = YX_panel_raw.iloc[:, 1:split_column]
    for i in range(0,Y_etl.shape[0]):
        row = Y_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl.iloc[i,j] = sustitute[j]

    Y_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                     'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
    Y_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    Y_player_column = Y_etl.pop('Jugador')
    Y_etl.insert(0, 'Jugador', Y_player_column)
    Y_etl_copy = Y_etl.copy()
    y_etl_names = Y_etl_copy['Jugador']
    for i in range(0,len(y_etl_names)):
        row = Y_etl_copy.iloc[i]
        row_length = row.size
        k = 0

        sustitute = []

        for j in range(0, row_length):
            if pd.isna(row[j]) != True:
                k = k + 1

        sustitute.append(y_etl_names[i])
        for j in range(1,k - 1):
            sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl_copy.iloc[i,j] = sustitute[j]

    Y_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                          'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
    Y_etl = Y_etl_copy

    # Base de datos
    regression_list = [None]*(period-1)
    for i in range(0,len(regression_list)):
        merge = pd.merge(Y_etl.iloc[:,[0,i+1]], X_etl.iloc[:,[0,i+1]])
        regression_list[i] = merge
        regression_list[i].columns = ['Jugador', 'Y', 'X']
    df_article_regression = regression_list[0]
    for i in range(1,len(regression_list)):
        df_article_regression = pd.concat([df_article_regression, regression_list[i]])
    df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    df_article_regression.dropna(inplace = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    
    # Construimos la regresion
    Y = df_article_regression['Y'].tolist()
    X = df_article_regression['X'].tolist()
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    
    # Regresamos como argumento al modelo
    return [df_article_regression, model]

In [13]:
X_raw_pitcher_t = 'Inning_pitched'
Y_raw_t = 'Sueldo'
df_copy_t = df_pitchers_t_copy

In [14]:
df_regression_t, model_t = etl_regression(X_raw_pitcher_t, Y_raw_t, df_copy_t)

Veamos la base de datos ocupada para la regresión y después los datos sobre la regresión

In [15]:
df_regression_t

,Jugador,Y,X
0,A.J. Burnett,-471.040165,14.618481
1,Aaron Harang,647.699528,13.405223
2,Aaron Harang,-138.427417,-10.689863
3,Adam Ottavino,-368.340054,-2.042177
4,Adam Ottavino,-18.852808,-7.398706
...,...,...,...
420,Zack Greinke,6.483182,-13.528012
421,Zack Greinke,-75.049658,-13.113161
422,Zack Greinke,-64.819869,-10.286993
423,Zack Greinke,740.407614,-13.113161


In [16]:
model_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1909
Date:                Tue, 11 Oct 2022   Prob (F-statistic):              0.662
Time:                        11:24:58   Log-Likelihood:                -3447.7
No. Observations:                 425   AIC:                             6899.
Df Residuals:                     423   BIC:                             6907.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         27.9714     40.685      0.688      0.492     -51.999     107.941
x1            -2.0278      4.641     -0.437      0.662     -11.150       7.094
==============================================================================
Omnibus:                       89.562   Durbin-Watson:                   2.229
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              260.360
Skew:                          -0.983   Prob(JB):                     2.91e-57
Kurtosis:                       6.292   Cond. No.                         9.10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Vemos que el algoritmo funciona adecuadamente

## Pitchers

Hagamos una prueba con una serie de estadísticas deportivas para cada grupo

In [17]:
pitcher_statistics_t = ['Bateos_pitcher', 'Carreras', 'Carreras_ganadas', 'Comando', 'Control',
                        'Dominio', 'ERA', 'Inning_pitched', 'Juegos', 'Juegos_iniciados', 
                        'Losses', 'Saves', 'TVS', 'WAR', 'WHIP', 'Walks', 'Wins', 
                        'Bateos_pitcher_2', 'Carreras_2', 'Carreras_ganadas_2', 'Comando_2', 'Control_2',
                        'Dominio_2', 'ERA_2', 'Inning_pitched_2', 'Juegos_2', 'Juegos_iniciados_2', 
                        'Losses_2', 'Saves_2', 'TVS_2', 'WAR_2', 'WHIP_2', 'Walks_2', 'Wins_2']
Y_pitcher_raw_t = 'ln_Sueldo'
pitcher_Statistics_len = len(pitcher_statistics_t)
pitcher_model_t = [None]*pitcher_Statistics_len
df_pitcher_model_t = [None]*pitcher_Statistics_len

In [18]:
for statistic in range(0,pitcher_Statistics_len):
    df_pitcher_model_t[statistic], pitcher_model_t[statistic] = etl_regression(pitcher_statistics_t[statistic],
                                                                               Y_pitcher_raw_t,
                                                                               df_pitchers_t_copy)
    
    print(colored(pitcher_statistics_t[statistic], "cyan"))
    print(pitcher_model_t[statistic].summary())
    print('\n')

Bateos_pitcher
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.368
Date:                Tue, 11 Oct 2022   Prob (F-statistic):              0.243
Time:                        11:25:01   Log-Likelihood:                 307.69
No. Observations:                 425   AIC:                            -611.4
Df Residuals:                     423   BIC:                            -603.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0377      0.006     

## Hitters

Hagamos lo mismo para los *hitters*.

In [19]:
hitter_statistics_t = ['At-bats', 'Bateos', 'Home-runs', 'Juegos',
                       'OPS', 'Porcentaje_bateo', 'Porcetnaje_juegos',
                       'RBI', 'TVS', 'WAR', 'At-bats_2', 'Bateos_2',
                       'Home-runs_2', 'Juegos_2', 'OPS_2',
                       'Porcentaje_bateo_2', 'Porcetnaje_juegos_2',
                       'RBI_2', 'TVS_2', 'WAR_2']
Y_hitter_raw_t = 'ln_Sueldo'
hitter_Statistics_len = len(hitter_statistics_t)
hitter_model_t = [None]*hitter_Statistics_len
df_hitter_model_t = [None]*hitter_Statistics_len

In [20]:
for statistic in range(0,hitter_Statistics_len):
    df_hitter_model_t[statistic], hitter_model_t[statistic] = etl_regression(hitter_statistics_t[statistic],
                                                                             Y_hitter_raw_t, df_hitters_t_copy)
    
    print(colored(hitter_statistics_t[statistic], "cyan"))
    print(hitter_model_t[statistic].summary())
    print('\n')

At-bats
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.715
Date:                Tue, 11 Oct 2022   Prob (F-statistic):              0.100
Time:                        11:26:05   Log-Likelihood:                 348.75
No. Observations:                 492   AIC:                            -693.5
Df Residuals:                     490   BIC:                            -685.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0468      0.005     -8.598 